# Elasticsearch CURD

## 1. Create

### 1.1. Create index

In [ ]:
# create an index
echo -e "* create index as: ";

curl -H 'Cache-Control: no-cache' -H 'Content-Type: application/json' \
     -X PUT 'http://localhost:9200/person?pretty' -d '{
         "settings": {
            "number_of_replicas": 0,
            "number_of_shards": 1
        },
        "mappings": {
            "properties": {
                "name": {
                    "type": "keyword"
                },
                "gender": {
                    "type": "keyword"
                },
                "birthday": {
                    "type": "date"
                },
                "role": {
                    "type": "keyword"
                },
                "department": {
                    "properties": {
                        "college": {
                            "type": "keyword"
                        },
                        "program": {
                            "type": "keyword"
                        }
                    }
                }
            }
        }
     }';

### 1.2. Create entities

In [ ]:
# create person 'alvin'
echo -e "* create person 'Alvin':";
person='{
    "name": "Alvin",
    "gender": "M",
    "birthday": "1981-03-17",
    "role": "STUDENT",
    "department": {
        "college": "Computer science",
        "program": "Software engineering"
    }
}';
curl -H 'Cache-Control: no-cache' -H 'Content-Type: application/json' \
     -X POST 'http://localhost:9200/person/_doc/001?pretty' -d "$(echo $person)";
     
     
# create person 'Emma'
echo -e "\n* create person 'Emma':";
person='{
    "name": "Emma",
    "gender": "F",
    "birthday": "1985-03-29",
    "role": "STUDENT",
    "department": {
        "college": "Computer science",
        "program": "Network engineering"
    }
}';
curl -H 'Cache-Control: no-cache' -H 'Content-Type: application/json' \
     -X POST 'http://localhost:9200/person/_doc/002?pretty' -d "$(echo $person)";
     
     
# create teacher 'Lucy'
echo -e "\n* create teacher 'Lucy':";
person='{
    "name": "Lucy",
    "gender": "F",
    "birthday": "1956-09-06",
    "role": "TEACHER",
    "department": {
        "college": "Computer science",
        "program": "Software engineering"
    }
}';
curl -H 'Cache-Control: no-cache' -H 'Content-Type: application/json' \
     -X POST 'http://localhost:9200/person/_doc/003?pretty' -d "$(echo $person)";

     
# create teacher 'Arthur'
echo -e "\n* create teacher 'Arthur':";
person='{
    "name": "Arthur",
    "gender": "M",
    "birthday": "1972-08-13",
    "role": "TEACHER",
    "department": {
        "college": "Computer science",
        "program": "Network engineering"
    }
}';
curl -H 'Cache-Control: no-cache' -H 'Content-Type: application/json' \
     -X POST 'http://localhost:9200/person/_doc/004?pretty' -d "$(echo $person)";

## 2. Update

### 2.1. Update by id

In [ ]:
# update by id
echo -e "* update by id:";
person='{
    "name": "Arthur Li",
    "gender": "M",
    "birthday": "1972-08-13",
    "role": "TEACHER",
    "department": {
        "college": "Computer science",
        "program": "Network engineering"
    }
}';
curl -H 'Cache-Control: no-cache' -H 'Content-Type: application/json' \
     -X PUT 'http://localhost:9200/person/_doc/004?pretty' -d "$(echo $person)";

# query by id after updated
echo -e "\n* query after update:";
curl -H 'Cache-Control: no-cache' -H 'Content-Type: application/json' \
     -X GET 'http://localhost:9200/person/_doc/004?pretty';

### 2.2. Update by query condition

- Arguments:
    - `conflicts`: `conflicts=proceed`, make sure simply count version conflicts, and not cause the `_update_by_query` to abort;

In [ ]:
# update by query condtion
echo -e "* update by query condition:";
update='{
    "script": {
        "source": "ctx._source[\"gender\"] = \"F\"; ctx._source[\"department\"][\"program\"] = params.program;",
        "params": {
            "program": "Software engineering"
        },
        "lang":"painless"
    },
    "query": {
        "term": {
            "role": "TEACHER"
        }
    }
}';
curl -H 'Cache-Control: no-cache' -H 'Content-Type: application/json' \
     -X POST 'http://localhost:9200/person/_update_by_query?pretty&conflicts=proceed' -d "$(echo $update)";

# query by id after updated
echo -e "\n* query after update:";
curl -H 'Cache-Control: no-cache' -H 'Content-Type: application/json' \
     -X GET 'http://localhost:9200/person/_doc/_search?pretty&q=role:TEACHER';

## 3. Delete

### 3.1. Delete by id

In [ ]:
# delete entity by id
echo -e "* delete entity by id:";
curl -H 'Cache-Control: no-cache' -H 'Content-Type: application/json' \
     -X DELETE 'http://localhost:9200/person/_doc/002?pretty';

# query all after delete
echo -e "\n* after delete, entites are:";
curl -H 'Cache-Control: no-cache' -H 'Content-Type: application/json' \
     -X GET 'http://localhost:9200/person/_search?pretty';

### 3.2. Delete by query condtion

- Arguments:
    - `conflicts`: `conflicts=proceed`, make sure simply count version conflicts, and not cause the `_update_by_query` to abort;

In [ ]:
# delete by query condition
echo -e "* delete entity by id:";
condition='{
    "query": {
        "term": {
            "role": "STUDENT"
        }
    }
}';
curl -H 'Cache-Control: no-cache' -H 'Content-Type: application/json' \
     -X POST 'http://localhost:9200/person/_delete_by_query?pretty&conflicts=proceed' -d "$(echo $condition)";


# query all after delete
echo -e "\n* after delete, entites are:";
curl -H 'Cache-Control: no-cache' -H 'Content-Type: application/json' \
     -X GET 'http://localhost:9200/person/_search?pretty';

## 4. Query

### 4.1. Basic query

#### 4.1.1. Query all

In [ ]:
echo -e "* query all entities are: ";

curl -H 'Cache-Control: no-cache' -H 'Content-Type: application/json' \
     -X GET 'http://localhost:9200/person/_search?pretty';

#### 4.1.2. Query by id

In [ ]:
echo -e "* query entity by id '001' is: ";

curl -H 'Cache-Control: no-cache' -H 'Content-Type: application/json' \
     -X GET 'http://localhost:9200/person/_doc/001?pretty';

### 4.2. Query string query

- Parameters:
    - `q`: Lucene query string, see also [LuceneTutorial](http://www.lucenetutorial.com/lucene-query-syntax.html)
    - `other parameters`: see also [ES doc](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-uri-request.html)

#### 4.2.1. Query by `attr=value`

In [ ]:
echo -e "* query all 'role is STUDENT' entites are: ";
curl -H 'Cache-Control: no-cache' -H 'Content-Type: application/json' \
     -X GET 'http://localhost:9200/person/_doc/_search?pretty&q=role:STUDENT';

#### 4.2.2. Query by `attr.subattr like value`

In [ ]:
echo -e "* query all 'department.program like Software' entites are: ";
curl -H 'Cache-Control: no-cache' -H 'Content-Type: application/json' \
     -X GET 'http://localhost:9200/person/_doc/_search?pretty&q=department.program:Software*';

#### 4.2.3. Query by `attr1=value1 AND attr2=value2`

- notice:
    - URL must be encoded. eg: blank space must be replaced by `%20`

In [ ]:
echo -e "* query all 'role is STUDENT AND department.program like Software' entites are: ";
curl -H 'Cache-Control: no-cache' -H 'Content-Type: application/json' \
     -X GET 'http://localhost:9200/person/_doc/_search?pretty&q=role:STUDENT%20AND%20department.program:Software*';

#### 4.2.4. Query by `attr1=value1 OR attr2=value2`

In [ ]:
echo -e "* query all 'role is STUDENT OR department.program like Software' entites are: ";
curl -H 'Cache-Control: no-cache' -H 'Content-Type: application/json' \
     -X GET 'http://localhost:9200/person/_doc/_search?pretty&q=role:STUDENT%20OR%20department.program:Software*';

#### 4.2.5. Query by `attr1=value1 AND attr2!=value2`

In [ ]:
echo -e "* query all 'role is STUDENT AND department.program NOT like Software' entites are: ";
curl -H 'Cache-Control: no-cache' -H 'Content-Type: application/json' \
     -X GET 'http://localhost:9200/person/_doc/_search?pretty&q=role:STUDENT%20-(department.program:Software*)';

### 4.2. Query DSL

### 

### 

### 

### 

### 

### 

In [ ]:
echo -e "* delete index as:";

curl -H 'Cache-Control: no-cache' -H 'Content-Type: application/json' \
     -X DELETE 'http://localhost:9200/person?pretty';